##  Generate json file with test data

> Here you want to generate a json file containing data you will be able to use to call your service  
> **You can skip / remove this step if you already have a json file with test data**  
> To use your file, simply copy it to **testdata folder** eg. testdata/european.json  
> **Data should not require any further procesing before calling the service**  

In [3]:
# retrieve apple listed options contracts from yahoo using yfinance library
# format data and save it as a json file

import os
import datetime 
import pandas as pd
import yfinance as yf

# pandas df which will contain puts and calls option quotes for all expiries
flat_df = pd.DataFrame()

# retrieve data for AAPL (Apple)
aapl = yf.Ticker("AAPL")

# main loop where we fill flat_df
for expi in aapl.options:

    # get option chain df for given expiry
    chain = aapl.option_chain(expi)
    
    # add putcall column to both calls and puts df
    chain.calls["putcall"] = "call"
    chain.puts["putcall"] = "put"

    # append both calls and puts df to flat_df and add expiry column
    flat_df = flat_df.append(chain.calls, sort=False)
    flat_df = flat_df.append(chain.puts, sort=False)
    flat_df["expiry"] = datetime.datetime.strptime(expi, '%Y-%m-%d')


# add undticker column, filter columns and rename contractSymbol column
flat_df["undticker"] = "AAPL"
flat_df = flat_df[["contractSymbol", "undticker", "expiry", "strike", "putcall"]]
flat_df = flat_df.rename(columns={'contractSymbol': 'ticker'})

# save flat_df to ./yahoo.json
res = flat_df.to_json(os.path.join(os.getcwd(),"../testdata/european.json"), orient='records', date_format='epoch')

print flat_df.head(10)
if res is not None:
    print 'an error occured'
    print res

                ticker undticker     expiry  strike putcall
0  AAPL200103C00200000      AAPL 2019-12-13   200.0    call
1  AAPL200103C00220000      AAPL 2019-12-13   220.0    call
2  AAPL200103C00225000      AAPL 2019-12-13   225.0    call
3  AAPL200103C00235000      AAPL 2019-12-13   235.0    call
4  AAPL200103C00237500      AAPL 2019-12-13   237.5    call
5  AAPL200103C00240000      AAPL 2019-12-13   240.0    call
6  AAPL200103C00242500      AAPL 2019-12-13   242.5    call
7  AAPL200103C00245000      AAPL 2019-12-13   245.0    call
8  AAPL200103C00247500      AAPL 2019-12-13   247.5    call
9  AAPL200103C00250000      AAPL 2019-12-13   250.0    call


## Call service with data from a json file

In [5]:
import simplejson as json
import requests

# load test data from json file and call service endpoint

contracts = json.load(open(os.path.join(os.getcwd(),"../testdata/european.json")))

res = [requests.post('http://localhost:3000/europeans', json=contract) for contract in contracts]

print res[:10]

ConnectionError: HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /europeans (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x119190f90>: Failed to establish a new connection: [Errno 61] Connection refused',))

## Check

In [ ]:
# retrieve data from service and print

res = requests.get('http://localhost:3000/europeans')
europeans = res.json()['europeans']

print json.dumps(europeans[:20], indent=4)